# Task 4 predictions

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import *
import matplotlib as mpl 
mpl.rcParams['figure.dpi'] = 300    
from sklearn.metrics import make_scorer

In [2]:
#generic input training
train_descriptors = pd.read_csv("train_descriptors.csv")
train_mord3d = pd.read_csv("train_mord3d.csv")
train_morgan = pd.read_csv("train_morgan.csv")
train_rdk = pd.read_csv("train_rdk.csv")

#crystal structures
train_crystals = pd.read_csv("train_crystals.csv")
#atom distance data in crystal structures
train_distances = pd.read_csv("train_distances.csv")
#centroid distance data in crystal structures
train_centroid_distances = pd.read_csv("train_centroid_distances.csv")

test_descriptors = pd.read_csv("test_descriptors.csv")
test_mord3d = pd.read_csv("test_mord3d.csv")
test_morgan = pd.read_csv("test_morgan.csv")
test_rdk = pd.read_csv("test_rdk.csv")

#need to predict density of structure

#centrosymmetric, and density is here
train_crystals.keys()
train_centrosymmetric = train_crystals["is_centrosymmetric"]
train_density = train_crystals["calculated_density"]

#van der Waal contacts is in train distances

train_distances.keys()
train_VDW = train_distances["n_vdw_contacts"]

#centroid distances are here

train_centroid_distances.keys()

train_centroid = train_centroid_distances["mean"]

train_descriptors_full = train_descriptors.iloc[:, 3:-2].dropna(axis= 1, how="any")
test_descriptors_full = test_descriptors[train_descriptors_full.columns]

In [144]:
dropable_columns = []

In [145]:
#need to predict density of structure

#train_centrosymmetric
#train_density
#train_VDW 
#train_centroid 

In [146]:
train_descriptors_full = train_descriptors.iloc[:, 3:-2].dropna(axis= 1, how="any")
test_descriptors_full = test_descriptors[train_descriptors_full.columns]

for i in train_descriptors_full:
    N_uniques = train_descriptors_full[i].unique()
    if len(N_uniques) < 2:
        dropable_columns.append(i)

train_descriptors_full = train_descriptors_full.drop(columns=dropable_columns)
test_descriptors_full = test_descriptors_full.drop(columns=dropable_columns)

In [147]:
from sklearn.model_selection import train_test_split

X_train_final,X_test_final,y_train_final,y_test_final=  train_test_split(
            train_descriptors_full,train_VDW, test_size = 0.2,
            random_state= 42)

In [149]:
ss = preprocessing.StandardScaler()
ss.fit(X_train_final)
X_train_ss = ss.transform(X_train_final)
X_test_ss = ss.transform(X_test_final)

In [150]:
import torch

In [151]:
all_data =  list((zip(np.array(X_train_ss).astype(dtype=float),
                              np.array(y_train_final).astype(dtype=float))))   
data_loader = torch.utils.data.DataLoader(all_data,
                                          batch_size=100,
                                          shuffle=True)

In [152]:
from torch import nn

In [153]:
import torch.nn.functional as F

In [154]:
# -*- coding: utf-8 -*-
import torch
import math


class MyNetwork(nn.Module):
    def __init__(self):
        # call constructor from superclass
        super().__init__()
        
        # define network layers
        self.fc1 = nn.Linear(891, 100)
        self.fc2 = nn.Linear(100,10)
        self.fc3 = nn.Linear(10,1)
        self.dropout = nn.Dropout(0.25)
    def forward(self, x):
        # define forward pass
        x = self.fc1(x)
        x = F.relu(self.fc2(x))
        x  = self.dropout(x)
        x = F.relu(self.fc3(x))
        return x

# Construct our model by instantiating the class defined above
model = MyNetwork()

In [155]:
model = model.float()

In [156]:
import torch.optim as optim

criterion = nn.L1Loss()
optimizer = optim.Adam(model.parameters(), lr=0.001,)

In [158]:
#we manually did early stopping at Epoch 272.

In [134]:
for epoch in range(20000):  # loop over the dataset multiple times

    for i, data in enumerate(data_loader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs.float())[:,0]
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        if epoch % 10 == 1 and i == 0:    # print every 2000 mini-batches
            model.eval()
            predictions = np.array(model(torch.from_numpy(X_train_ss).float()).detach())[:,0]
            in_sample_loss  = np.mean(np.abs(predictions-y_train_final))
            
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, in_sample_loss ))
            predictions = np.array(model(torch.from_numpy(X_test_ss).float()).detach())[:,0]
            print ("%.3f"%(np.mean(np.abs(predictions-y_test_final)) ,) )
            model.train()
            
print('Finished Training')

[2,     1] loss: 5.078
5.229
[12,     1] loss: 4.008
4.235
[22,     1] loss: 3.841
4.121
[32,     1] loss: 3.763
4.033
[42,     1] loss: 3.736
4.007
[52,     1] loss: 3.666
3.968
[62,     1] loss: 3.728
4.040
[72,     1] loss: 3.654
3.955
[82,     1] loss: 3.636
3.974
[92,     1] loss: 3.619
3.980
[102,     1] loss: 3.572
3.938
[112,     1] loss: 3.588
3.970
[122,     1] loss: 3.537
3.963
[132,     1] loss: 3.522
3.935
[142,     1] loss: 3.509
3.932
[152,     1] loss: 3.533
3.991
[162,     1] loss: 3.476
3.960
[172,     1] loss: 3.529
3.974
[182,     1] loss: 3.474
3.939
[192,     1] loss: 3.445
3.936
[202,     1] loss: 3.432
3.938
[212,     1] loss: 3.447
3.959
[222,     1] loss: 3.405
3.906
[232,     1] loss: 3.425
3.955
[242,     1] loss: 3.393
3.936
[252,     1] loss: 3.410
3.965
[262,     1] loss: 3.414
3.967
[272,     1] loss: 3.367
3.973


KeyboardInterrupt: 

In [159]:
model.eval()

predictions = np.array(model(torch.from_numpy(X_train_ss).float()).detach())[:,0]

np.mean(np.abs(predictions-y4_train_final))

predictions = np.array(model(torch.from_numpy(ss.transform(test_descriptors_full)).float()).detach())

np.savetxt("task_4_predictions_final.csv", predictions)